In [ ]:
import json

In [3]:
athlete_file = None
activity_detail_file = None
activity_streams_file = None

In [ ]:
with open(athlete_file, 'r') as f:
    athlete = json.load(f)

with open(activity_detail_file, 'r') as f:
    activity_detail = json.load(f)

with open(activity_streams_file, 'r') as f:
    activity_streams = json.load(f)

In [ ]:
print(f'athlete firstname: {athlete["firstname"]}')
print(f'activity_detail: {activity_detail["external_id"]}')
print(f'activity_streams: {activity_streams["altitude"]}')